# Compare results from the Token Classification with other sources
Getting the data from the repo: https://github.com/JUrban/extract-defs/tree/main/lm-inputs/defsCT

In [57]:
import re
import os
from os.path import join
import json

from transformers import (AutoTokenizer,
                         TFAutoModelForTokenClassification,)

import ner.llm_utils as llu

cfg = {'checkpoint': 'bert-base-uncased',
      'max_length': 150, # check mp_infer_HFTrans_ner.py
      }

In [3]:
xdefs_root = join(os.path.expanduser('~'),'zornMedia/extract-defs')
xdefs_inputs = join(xdefs_root, 'lm-inputs/defsCT')
xdefs_outputs = join(xdefs_root, 'lm-outputs/defsCT')
xdefs_inputs_filelst = sorted(os.listdir(xdefs_inputs))
xdefs_outputs_filelst = sorted(os.listdir(xdefs_outputs))

with open(join(xdefs_outputs, xdefs_outputs_filelst[0]), 'r') as fobj:
    xdefs_out_lst = fobj.readlines()
with open(join(xdefs_inputs, xdefs_inputs_filelst[0]), 'r') as fobj:
    xdefs_in_lst = fobj.readlines()

In [37]:
# Example to grab 'Definition 8.1: DEFINES "taxotopic"\n'
# and 'Definition 7.2: DEFINES "taxotopic", "S-taxotopic", "taxotopy equivalent", "S-taxotopy equivalent", "null-taxotopic"\n'
#reg_expr = re.compile('Definition[\s\d\.\:]+DEFINES\s+"(.+?)"+')
reg_expr = re.compile('"(.+?)"+')
def get_term(out_str):
    # The out_str is the output string produced by the LLM
    Defin = reg_expr.findall(out_str)
    return Defin

reg_expr2 = re.compile('Definition\s+[\d\.]+\s+(.+)')
def get_text(in_str):
    Defin = reg_expr2.findall(in_str)
    return Defin[0]
get_text(xdefs_in_lst[5])    

'Suppose we are given two $n$-tuples of complementary orders $\\Xi_{1}$ and $\\Xi_{2}$ on the same set $X.$ We will say that $\\Xi_{1}$ dominates $\\Xi_{2}$ (notation $\\Xi_{2}\\triangleleft\\Xi_{1}$) if $i<_{p}j$ in $\\Xi_{1}$ implies $i<_{r}j$ for some $r\\geq p$ or $j<_{r}i$ for $r>p$ in $\\Xi_{2}.$'

In [39]:
reg_expr3 = re.compile('\$.+?\$')
def remove_latex_formulas(text):
    return re.sub(reg_expr3,'_inline_math_', text)
remove_latex_formulas(get_text(xdefs_in_lst[24]))

'A dg-vector bundle over a formal pointed dg-manifold _inline_math_ is given by a linearly compact vector bundle _inline_math_ over _inline_math_ such that the corresponding module _inline_math_ carries a differential _inline_math_ so that _inline_math_ becomes a dg-module over the dg-algebra _inline_math_ and _inline_math_ vanishes on _inline_math_.'

In [4]:
tokenizer = AutoTokenizer\
          .from_pretrained(cfg['checkpoint'])

In [5]:
tokenizer(xdefs_in_lst[15][:100],
         return_tensors='tf',
         is_split_into_words=False,
         padding=True,
         truncation=True,
         max_length=cfg['max_length'])

2024-08-05 03:42:05.173600: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-05 03:42:05.175761: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-05 03:42:05.183593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 03:42:05.197704: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 03:42:05.201749: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 03:42:05.212345: I tensorflow/core/platform/cpu_feature_gu

{'input_ids': <tf.Tensor: shape=(1, 31), dtype=int32, numpy=
array([[  101,  6210,  1023,  1012,  1017,  1037,  4359,  8040,  2828,
         1002,  1050,  1002,  1011,  3850,  2094,  1999,  1037, 19490,
        18847, 16975,  4696,  1002,  1058,  1002,  2003,  1037,  3074,
         1002,  1037,  1035,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 31), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 31), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [9]:
xdefs_in_lst[15]

'Definition 9.3 A reduced SC type $n$-operad in a symmetric monoidal category $V$ is a collection $A_{T}\\in V$ where $T$ runs over all coloured pruned $n$-trees such that $A_{U^{1}_{n}}=A_{U^{2}_{n}}=I$ and $A_{z^{n}U_{0}}=I,$ where $U^{1}_{n}$ is a linear tree with its unique tip coloured by $1$ and $U^{2}_{n}$ is a linear tree with its tip coloured by $2.$ This collection is equipped with a morphism $m_{\\sigma}:A_{S}\\otimes A_{T_{1}^{(p)}}\\otimes\\ldots\\otimes A_{T_{k}^{(p)}}% \\rightarrow A_{T}$ for every coloured morphism of trees $\\sigma:T\\rightarrow S$ between coloured pruned $n$-trees. They must satisfy the obvious associativity and unitarity conditions. For an unbased reduced SC-operad we use collections without degenerate trees and define multiplication only with respect to surjections of coloured trees.\n'

In [49]:
get_term(xdefs_out_lst[3])

['Getzler-Jones $n$-operad']

In [44]:
out_dict = {'in_text': 'hola como estan?',
            'out_prompt': 'bien',
            'out_finetune': 'todo bien'}

json.dumps(out_dict)

'{"in_text": "hola como estan?", "out_prompt": "bien", "out_finetune": "todo bien"}'

In [45]:
os.path.isdir('/home/luis/zornMedia/')

True

In [47]:
assert False, 'hola'

AssertionError: hola

In [51]:
ll = []
ll[0] if len(ll)>0 else None

In [56]:
concat_tokens=[['[CLS]', 'The', 'Get', '##zle', '##r', '-', 'Jones', '_', 'in', '##line', '_', 'math', '_', '-', 'opera', '##d', '_', 'in', '##line', '_', 'math', '_', '\\', 'b', '##f', 'G', '##J', '_', 'in', '##line', '_', 'math', '_', 'is', 'the', 'image', 'of', '_', 'in', '##line', '_', 'math', '_', 'This', 'is', 'a', 'reduced', '_', 'in', '##line', '_', 'math', '_', '-', 'terminal', '_', 'in', '##line', '_', 'math', '_', '-', 'opera', '##d', '.', '[SEP]']]
ccc =  ['O', 'O', 'B-DFNDUM', 'I-DFNDUM', 'I-DFNDUM', 'I-DFNDUM', 'I-DFNDUM', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DFNDUM', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
len(concat_tokens[0]), len(ccc)

(66, 66)

In [59]:
llu.crop_terms(concat_tokens[0], ccc)

KeyError: 'special_tokens'

In [63]:
f"_{4:0>3}"

'_004'